In [ ]:
pip install langchain groq faiss-cpu sentence-transformers PyPDF2 python-docx langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from PyPDF2 import PdfReader
from docx import Document

def extract_text(file_paths):
    documents = []
    for file_path in file_paths:
        if file_path.endswith('.pdf'):
            with open(file_path, 'rb') as file:
                pdf_reader = PdfReader(file)
                text = "".join(page.extract_text() or "" for page in pdf_reader.pages)
                documents.append({"content": text, "source": file_path})
        elif file_path.endswith('.docx'):
            doc = Document(file_path)
            text = "\n".join(para.text for para in doc.paragraphs)
            documents.append({"content": text, "source": file_path})
    return documents

file_path = ["/content/DeepSeek_R1.pdf" ]
text = extract_text(file_path)
print(text)

[{'content': 'DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via\nReinforcement Learning\nDeepSeek-AI\nresearch@deepseek.com\nAbstract\nWe introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.\nDeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-\nvised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.\nThrough RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing\nreasoning behaviors. However, it encounters challenges such as poor readability, and language\nmixing. To address these issues and further enhance reasoning performance, we introduce\nDeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-\nR1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the\nresearch community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models\n(1.5B, 7B, 8B, 14B, 32B, 70B) distil

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    chunks = []
    for doc in documents:
        split_docs = text_splitter.create_documents(
            [doc["content"]],
            metadatas=[{"source": doc["source"]}]
        )
        chunks.extend(split_docs)
    return chunks

chunks = chunk_text(text)
for i, chunk in enumerate(chunks, 1):
    print(f"\n🔹 **Chunk {i}:**\n{chunk.page_content}\n{'-'*50}")


🔹 **Chunk 1:**
DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
vised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.
Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behaviors. However, it encounters challenges such as poor readability, and language
mixing. To address these issues and further enhance reasoning performance, we introduce
DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-
R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the
research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models
------------------------------------------------

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

def generate_embeddings(chunks):
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    return embedding_model, chunks

embedding_model, chunks = generate_embeddings(chunks)

<ipython-input-6-61f3aaff5793>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import FAISS

def store_in_db(chunks, embedding_model):
    vector_db = FAISS.from_documents(chunks, embedding_model)
    return vector_db

vector_db = store_in_db(chunks, embedding_model)

In [ ]:
def query_to_embeddings(query, embedding_model):
    query_embedding = embedding_model.embed_query(query)
    return query_embedding

query = "what is distilation?"
query_embedding = query_to_embeddings(query, embedding_model)


In [ ]:
def retrieve_top_matches(vector_db, query_embedding):
    top_matches = vector_db.similarity_search_by_vector(query_embedding, k=3)
    return top_matches

top_matches = retrieve_top_matches(vector_db, query_embedding)
print(top_matches)

[Document(id='a497e44f-ee3b-4998-b373-1d0569b2187a', metadata={'source': '/content/DeepSeek_R1.pdf'}, page_content='reasoning-related benchmarks.\nAs shown in Table 5, simply distilling DeepSeek-R1’s outputs enables the efficient DeepSeek-\nR1-7B (i.e., DeepSeek-R1-Distill-Qwen-7B, abbreviated similarly below) to outperform non-\nreasoning models like GPT-4o-0513 across the board. DeepSeek-R1-14B surpasses QwQ-32B-\nPreview on all evaluation metrics, while DeepSeek-R1-32B and DeepSeek-R1-70B significantly\nexceed o1-mini on most benchmarks. These results demonstrate the strong potential of distilla-\ntion. Additionally, we found that applying RL to these distilled models yields significant further\ngains. We believe this warrants further exploration and therefore present only the results of the\nsimple SFT-distilled models here.\n4. Discussion\n4.1. Distillation v.s. Reinforcement Learning\nIn Section 3.2, we can see that by distilling DeepSeek-R1, the small model can achieve impressiv

In [ ]:
from groq import Groq

def pass_to_llm(query, top_matches, api_key):
    context = "\n\n".join(match.page_content for match in top_matches)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    client = Groq(api_key=api_key)
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="gemma2-9b-it"
    )
    return chat_completion.choices[0].message.content, top_matches


api_key="your_api_key"
llm_response, top_matches = pass_to_llm(query, top_matches, api_key)
print(llm_response)

Based on the context you provided, **distillation** is a method for transferring knowledge from a larger, more powerful model (like DeepSeek-R1) to a smaller model. 

Here's how it works:

1. **Training:** A large model is trained on a dataset of reasoning tasks.
2. **Knowledge Extraction:**  The outputs (predictions or representations) of the large model are used to train a smaller model.
3. **Smaller Model Enhancement:** The smaller model learns to mimic the behavior of the larger model, effectively acquiring its reasoning capabilities.

The result is a smaller model that performs razoning tasks comparably to the larger model, but is more efficient to run.


Let me know if you have any other questions!

